### Download html files from bis.org central banker speech site

In [86]:
import os
import logging
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
driver = webdriver.Chrome('./chromedriver')

In [3]:
#Access BIS speech site
driver.get("https://www.bis.org/cbspeeches/index.htm?m=2%7C10")

In [53]:
tot_page = driver.find_elements(By.XPATH, "//div[@class='pageof']")[0]
tot_page = tot_page.find_elements_by_css_selector("*")
tot_page = int(bar2.text.split(' ')[-1].replace(',', ''))
tot_page

1601

In [53]:
from pathlib import Path

#download folder
down_folder = r'..\data\bisorg_cbspeech'
Path(down_folder).mkdir(parents=True, exist_ok=True)

In [63]:
for i in range(tot_page):    
    _datapage = driver.find_elements(By.XPATH, "//a[@data-page]")[-1]
    _html = str.encode(driver.page_source)
    _filename = 'p{0}.html.txt'.format(10000+i)
    with open(os.path.join(down_folder,_filename),'wb') as fout:
        fout.write(_html)
    _datapage.click()
    time.sleep(3)
    if i % 100 == 0:
        print('{0}th page downloaded'.format(i))

0th page downloaded


#### Extract link from above html files

In [2]:
import bs4
import os
import glob

In [8]:
htmlfiles=os.listdir(down_folder)

In [87]:
def generate_speech_list(htmlfile):
    
    soup = bs4.BeautifulSoup(open(os.path.join(down_folder,htmlfiles[0]), 'rb').read().decode())
    content = soup.find("table",{"class":"documentList"})
    
    speeches = list()
    for i in content.find_all('tr'):
        try:
            dict_speech = dict()
            dict_speech['date'] = i.find("td",{"class":"item_date"}).text.strip()
            dict_speech['link'] = i.find("a")['href'].strip()
            dict_speech['title'] = i.find("a").text.strip()
            dict_speech['info'] = i.find("div",{"class","info"}).text.strip().split('\n')[0]
        except Exception as e:
            print('Error({0}): {1}'.format(i,e))
            break
        speeches.append(dict_speech)
    
    return speeches

In [50]:
dict_speech = dict()
for htmlfile in htmlfiles:
    dict_speech[htmlfile] = generate_speech_list(htmlfile)

#### Download a speech html

In [55]:
import requests

In [57]:
url_bis = r"https://www.bis.org"

In [82]:
def extract_meta(meta_tags):
    dict_meta = dict()
    for i in meta_tags:
        if i.has_attr('name'):
            if i['name'] == 'viewport': #It is not related to Content meta
                continue
            else:
                dict_meta[i['name']] = i['content']
    return dict_meta

In [83]:
for _key in dict_speech:
    for _speech in dict_speech[_key]:
        soup = bs4.BeautifulSoup(requests.get(url_bis+_speech['link']).content.decode()) 
        _speech['meta'] = extract_meta(soup.find_all("meta"))
        _speech['content'] = soup.find("div",{"id":"cmsContent"}).text.split('\n')